In [ ]:
from PIL import Image
from torchvision import transforms as T

img = Image.open("./nerf_synthetic/lego/train/r_0.png")
img = img.resize((400,400), Image.LANCZOS)
img = T.ToTensor()(img)
print(img.shape)
# valid_mask = (img[-1]>0).flatten()
valid_mask = (img[-1]>0)
print(img.permute(1,2,0)[valid_mask]*256)
print(valid_mask.sum())
img = img.view(4,-1).permute(1,0)
print(img.shape)
img = img[:, :3]*img[:, -1:] + (1-img[:, -1:])
img = img.view(400,400,3)
print(img.shape)
print(img[valid_mask]*256)
# img = img.permute(1,0).view(3, 400, 400)
# img = T.ToPILImage()(img)
# img.save('test.png')

In [ ]:
from scipy.spatial import transform
import numpy as np
def blender_quat2rot(quaternion):
  """Convert quaternion to rotation matrix.
  Equivalent to, but support batched case:
  ```python
  rot3x3 = mathutils.Quaternion(quaternion).to_matrix()
  ```
  Args:
    quaternion:
  Returns:
    rotation matrix
  """

  # Note: Blender first cast to double values for numerical precision while
  # we're using float32.
  q = np.sqrt(2) * quaternion

  q0 = q[..., 0]
  q1 = q[..., 1]
  q2 = q[..., 2]
  q3 = q[..., 3]

  qda = q0 * q1
  qdb = q0 * q2
  qdc = q0 * q3
  qaa = q1 * q1
  qab = q1 * q2
  qac = q1 * q3
  qbb = q2 * q2
  qbc = q2 * q3
  qcc = q3 * q3

  # Note: idx are inverted as blender and numpy convensions do not
  # match (x, y) -> (y, x)
  rotation = np.empty((*quaternion.shape[:-1], 3, 3), dtype=np.float32)
  rotation[..., 0, 0] = 1.0 - qbb - qcc
  rotation[..., 1, 0] = qdc + qab
  rotation[..., 2, 0] = -qdb + qac

  rotation[..., 0, 1] = -qdc + qab
  rotation[..., 1, 1] = 1.0 - qaa - qcc
  rotation[..., 2, 1] = qda + qbc

  rotation[..., 0, 2] = qdb + qac
  rotation[..., 1, 2] = -qda + qbc
  rotation[..., 2, 2] = 1.0 - qaa - qbb
  return rotation

def make_transform_matrix(positions,rotations,):
  """Create the 4x4 transformation matrix.
  Note: This function uses numpy.
  Args:
    positions: Translation applied after the rotation.
      Last column of the transformation matrix
    rotations: Rotation. Top-left 3x3 matrix of the transformation matrix.
  Returns:
    transformation_matrix:
  """
  # Create the 4x4 transformation matrix
  rot_pos = np.broadcast_to(np.eye(4), (*positions.shape[:-1], 4, 4)).copy()
  rot_pos[..., :3, :3] = rotations
  rot_pos[..., :3, 3] = positions
  return rot_pos

def from_position_and_quaternion(positions, quaternions, use_unreal_axes):
    if use_unreal_axes:
      rotations = transform.Rotation.from_quat(quaternions).as_matrix()
    else:
      # Rotation matrix that rotates from world to object coordinates.
      # Warning: Rotations should be given in blender convensions as
      # scipy.transform uses different convensions.
      rotations = blender_quat2rot(quaternions)
    px2world_transform = make_transform_matrix(positions=positions,rotations=rotations)
    return px2world_transform


In [ ]:
import json
import glob
import pandas as pd
import numpy as np
with open("/home/timothy/Desktop/2023Spring/GeoNeRF/data/data/nesf_data/klevr/0/metadata.json","r") as f:
    file = json.load(f)
    # print(file['metadata']) # {'width': 256, 'height': 256, 'num_frames': 301, 'seed': 0}
    # print(file['camera'].keys()) # dict_keys(['focal_length', 'sensor_width', 'field_of_view', 'positions', 'quaternions', 'K', 'R', 'height', 'width'])
    # print(file['segmentation_labels']) # ['Cube', 'Cylinder', 'Sphere', 'Torus', 'Gear']
    # print(file['scene_boundaries']) # {'min': [-3.1, -3.1, -0.1], 'max': [3.1, 3.1, 3.1]}
    # print(file['split_ids']) # {"train": [0, 1], "test": [2]}} ...

positions = np.array(file['camera']['positions'])     # (301, 3)
quaternions = np.array(file['camera']['quaternions']) # (301, 4)
focal_px_length = file['camera']['focal_length'] * file['metadata']['width'] / file['camera']['sensor_width']

# print(positions.shape)
# print(quaternions.shape)

px2world_transform = from_position_and_quaternion(positions, quaternions, False)
print(px2world_transform.shape)                         # (301,4,4)
# print(file['camera']['K'])
# print(file['camera']['R'])


In [22]:
import PIL.Image as Image
from torchvision import transforms as T
import torch
import numpy as np

# label_list = []
# for i in range(200):
#     img = Image.open(f"/home/timothy/Desktop/2023Spring/GeoNeRF/data/data/nesf_data/klevr/{i}/segmentation_00010.png")
#     # img = Image.open("/home/timothy/Desktop/2023Spring/GeoNeRF/data/data/nesf_data/klevr/1/rgba_00000.png")
#     labels = np.unique(np.array(img))
#     for label in labels:
#         if label not in label_list:
#             label_list.append(label)

# print(label_list)

img = Image.open(f"/home/timothy/Desktop/2023Spring/GeoNeRF/data/data/nesf_data/klevr/{i}/segmentation_00010.png")
img = img.resize((128,128), Image.Resampling.LANCZOS)
img = torch.from_numpy(np.array(img)).long().unsqueeze(-1)
print(img.shape)
print(img)
unique = torch.unique(img)
print(unique)

torch.Size([1, 128, 128])
tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]])
tensor([0, 1, 2, 3, 5])


In [17]:
x = [60, 84, 205, 159, 91, 242, 53, 42, 126, 135, 270, 154, 189, 22, 15, 98, 180, 227, 34, 72, 27, 89, 30, 178, 150, 122, 99, 86, 144, 225, 181, 231, 61, 51, 52, 177, 57, 211, 157, 47, 3, 20, 113, 77, 220, 79, 264, 148, 49, 244, 119, 300, 263, 78, 252, 275, 274, 166, 285, 102, 299, 44, 253, 104, 75, 110, 171, 37, 117, 254, 54, 209, 114, 192, 196, 296, 292, 229, 130, 195, 297, 96, 289, 188, 277, 160, 121, 259, 140, 124, 187]
x = sorted(x)
print(len(x))
print(x)

91
[3, 15, 20, 22, 27, 30, 34, 37, 42, 44, 47, 49, 51, 52, 53, 54, 57, 60, 61, 72, 75, 77, 78, 79, 84, 86, 89, 91, 96, 98, 99, 102, 104, 110, 113, 114, 117, 119, 121, 122, 124, 126, 130, 135, 140, 144, 148, 150, 154, 157, 159, 160, 166, 171, 177, 178, 180, 181, 187, 188, 189, 192, 195, 196, 205, 209, 211, 220, 225, 227, 229, 231, 242, 244, 252, 253, 254, 259, 263, 264, 270, 274, 275, 277, 285, 289, 292, 296, 297, 299, 300]
